# 전처리를 해보자
- 1. 일단 기본값은 전부 칼럼이 존재해야한다.
- 2. 추후 나머지 칼럼들을 삭제해보며 모델 성능을 파악해본다.
- 3. 해당과정을 빠르게 가능하도록 파이프라인을 구축한다.


In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('jung.csv')

In [23]:
df["사고시"] = df["사고일시"].str.split(" ").str.get(3)
df['사고시'] = df['사고시'].str.extract('(\d+)').astype(int)
df['월'] = df['사고일시'].str.extract('(\d+)월').astype(str)
df["도광역시"] = df["시군구"].str.split(" ").str.get(0)

bins = [0, 6, 12, 18, 24]
labels = ['dawn', 'morn', 'aftern', 'night']

df['하루시간구분'] = pd.cut(df['사고시'], bins=bins, labels=labels, right=False)
df['하루시간구분'] = df['하루시간구분'].astype('object')

In [24]:
df['타겟'] = df['가해운전자 상해정도'].map({'경상': 1, '부상신고': 1, '기타불명': 0, '상해없음': 0, '사망': 2, '중상': 2})
df.drop('가해운전자 상해정도', inplace=True, axis=1)

In [25]:
dele = ['사고일시','사고번호','시군구']
# dele = ['사고일시','사고번호','시군구','사망자수','중상자수','경상자수','부상신고자수']
df.drop(columns=dele, inplace=True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30373 entries, 0 to 30372
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   요일          30373 non-null  object
 1   사고내용        30373 non-null  object
 2   사망자수        30373 non-null  int64 
 3   중상자수        30373 non-null  int64 
 4   경상자수        30373 non-null  int64 
 5   부상신고자수      30373 non-null  int64 
 6   사고유형        30373 non-null  object
 7   법규위반        30373 non-null  object
 8   노면상태        30373 non-null  object
 9   기상상태        30373 non-null  object
 10  도로형태        30373 non-null  object
 11  가해운전자 차종    30373 non-null  object
 12  가해운전자 성별    30373 non-null  object
 13  가해운전자 연령    30373 non-null  int64 
 14  피해운전자 차종    30373 non-null  object
 15  피해운전자 성별    30373 non-null  object
 16  피해운전자 연령    30373 non-null  int64 
 17  피해운전자 상해정도  30373 non-null  object
 18  사고시         30373 non-null  int32 
 19  월           30373 non-null  object
 20  도광역시  

In [28]:
df['타겟'].value_counts(normalize=True)

타겟
1    0.530010
0    0.290389
2    0.179600
Name: proportion, dtype: float64

In [17]:
# object 타입인 열들을 선택
object_columns = df.select_dtypes(include=['object']).columns
# 해당 열들에 대해서 원핫 인코딩 적용
df = pd.get_dummies(df, columns=object_columns,dtype=int)


In [18]:
df.head()

,사망자수,중상자수,경상자수,부상신고자수,가해운전자 연령,피해운전자 연령,사고시,타겟,요일_금요일,요일_목요일,...,도광역시_인천광역시,도광역시_전라남도,도광역시_전라북도,도광역시_제주특별자치도,도광역시_충청남도,도광역시_충청북도,하루시간구분_aftern,하루시간구분_dawn,하루시간구분_morn,하루시간구분_night
0,0,0,1,0,70,50,17,1,0,1,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,20,42,18,2,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,75,81,11,1,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,76,75,12,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,1,0,61,29,10,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
df.to_csv('prepro_jung.csv', index=False)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30373 entries, 0 to 30372
Columns: 119 entries, 사망자수 to 하루시간구분_night
dtypes: int32(112), int64(7)
memory usage: 14.6 MB
